In [48]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

In [49]:
file_path = "label_data/label_text_Adrian.csv"
df = pd.read_csv(file_path, header = None)
df[0][0]

'Boneless Lamb Product of Australia L Leg-Chump On-Shank Off IW/VAC Slaughter Date: 09 Jul 2024 Packed On: 10 Jul 2024 Best Before: 09 Jul 2026 Keep Frozen Pieces 5 13.28 Kg Net Weight Australia 686 Inspected'

In [50]:
import spacy
import csv

# Function to download the SpaCy model if not present
def download_spacy_model(model_name):
    try:
        nlp = spacy.load(model_name)
    except OSError:
        from spacy.cli import download
        download(model_name)
        nlp = spacy.load(model_name)
    return nlp

# Load SpaCy model
nlp = download_spacy_model("en_core_web_sm")

In [51]:
# Load SpaCy model
nlp = spacy.load("en_core_web_sm")

# Function to extract relevant information
def extract_information(text):
    doc = nlp(text)
    food_name = ""
    packaged_date = ""
    expiry_date = ""
    for ent in doc.ents:
        if ent.label_ == "DATE":
            if "pack" in text.lower():
                packaged_date = ent.text
            elif "best before" in text.lower() or "use by" in text.lower():
                expiry_date = ent.text
        elif ent.label_ == "PRODUCT":
            food_name = ent.text
    return food_name, packaged_date, expiry_date

# Read the CSV file
with open("label_data/label_text_Adrian.csv", mode='r', encoding='utf-8') as file:
    csv_reader = csv.reader(file)
    with open("extracted_info.csv", mode='w', newline='', encoding='utf-8') as output_file:
        csv_writer = csv.writer(output_file)
        csv_writer.writerow(["Food Name", "Packaged Date", "Expiry Date"])
        for row in csv_reader:
            text = row[0]
            food_name, packaged_date, expiry_date = extract_information(text)
            csv_writer.writerow([food_name, packaged_date, expiry_date])

print("Information extraction completed.")

Information extraction completed.


In [66]:
file_path = "label_data/label_text_Adrian.csv"
df = pd.read_csv(file_path, header = None)

i = 4
doc = nlp(df[0][1])
print(doc)
print(doc.ents[i])
print(doc.ents[i].label_)
doc.ents[i].label_ == "DATE"

Keep Frozen Product of Australia Beef Burger Bulk Pack Net Wt 13.63 kg 13.63 KG Pkg On 14 May 2024 Best Before 14 May 2025 Top Cut Foods Pty Ltd 101265 Boneless Beef NL Allergens
14 May 2024
DATE


True

In [52]:
extracted_data = pd.read_csv("extracted_info.csv")
extracted_data.head()

,Food Name,Packaged Date,Expiry Date
0,NaN,NaN,NaN
1,NaN,14 May 2025,NaN
2,NaN,NaN,3
3,NaN,NaN,NaN
4,NaN,NaN,72361


In [78]:
import spacy

# Load the SpaCy model
nlp = spacy.load("en_core_web_sm")

text = "Keep Frozen Product of Australia Beef Burger Bulk Pack Net Wt 13.63 kg 13.63 KG Pkg On 14 May 2024 Best Before 14 May 2025 Top Cut Foods Pty Ltd 101265 Boneless Beef NL Allergens"

doc = nlp(text)
food_name = ""
packaged_date = ""
expiry_date = ""

for ent in doc.ents:
    #print(f"{ent}, {ent.label_}")
    if ent.label_ == "DATE":
        start = ent.start
        end = ent.end
        
        # Get the tokens before and after the DATE entity
        before_ent = doc[start-3:start]
        after_ent = doc[end:end+3]
        
        # Check the context around the DATE entity
        context = before_ent.text.lower() + " " + after_ent.text.lower()

        # Define the phrases to look for
        packaging_phrases = ["packed", "pkg on", "packaged on"]
        expiry_phrases = ["best before", "use by"]

        if any(phrase in context for phrase in packaging_phrases):
            packaged_date = ent.text
        elif any(phrase in context for phrase in expiry_phrases):
            expiry_date = ent.text
    elif ent.label_ == "PERSON":
        food_name = ent.text

print(f"Example Text: {text}")
print(f"Food Name: {food_name}")
print(f"Packaged Date: {packaged_date}")
print(f"Expiry Date: {expiry_date}")

Example Text: Keep Frozen Product of Australia Beef Burger Bulk Pack Net Wt 13.63 kg 13.63 KG Pkg On 14 May 2024 Best Before 14 May 2025 Top Cut Foods Pty Ltd 101265 Boneless Beef NL Allergens
Food Name: KG Pkg
Packaged Date: 14 May 2024
Expiry Date: 14 May 2025
